In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
3,249,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,29/01/2023 19:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-11.931332,...,-11.931332+-69.11748,2023 19:30 HORAS+01+29+19:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
4,251,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,27/01/2023,29/01/2023 19:30 HORAS,MADRE DE DIOS/TAMBOPATA/TAMBOPATA,KM 425-426,(PUERTO MALDONADO) PARO INDEFINIDO SOLICITANDO...,-12.5905762,...,-12.5905762+-69.2221768,2023 19:30 HORAS+01+29+19:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
5,242,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,20/01/2023,29/01/2023 19:30 HORAS,MADRE DE DIOS/TAMBOPATA/LABERINTO,KM 382+200,(SECTOR LABERINTO) PARO INDEFINIDO SOLICITANDO...,-12.7637802,...,-12.7637802+-69.5700566,2023 19:30 HORAS+01+29+19:30,2,4,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
6,241,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,22/01/2023,29/01/2023 19:30 HORAS,MADRE DE DIOS/TAMBOPATA/INAMBARI,KM 347,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.892215,...,-12.892215+-69.8382055,2023 19:30 HORAS+01+29+19:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
7,240,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,22/01/2023,29/01/2023 19:30 HORAS,MADRE DE DIOS/TAMBOPATA/INAMBARI,KM 316,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.9015914,...,-12.9015914+-70.0846561,2023 19:30 HORAS+01+29+19:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,280,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,29/01/2023 19:30 HORAS,PUNO/CHUCUITO/ZEPITA,KM 287,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.6042637,...,-16.6042637+-69.1857072,2023 19:30 HORAS+01+29+19:30,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZO CRUZ - DESA...,SI,interrumpido
63,289,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,29/01/2023 19:30 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 310,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.72557,...,-16.72557+-69.67592,2023 19:30 HORAS+01+29+19:30,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZO CRUZ - DESA...,SI,interrumpido
64,285,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,29/01/2023 19:30 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 0,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.730059545472,...,-16.730059545472+-69.71403699189,2023 19:30 HORAS+01+29+19:30,0,0,CEOPOL,HUMANO,PE-38A,VIA MAZOCRUZ - ILAVE,SI,interrumpido
65,286,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,05/01/2023,29/01/2023 19:30 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 175 AL 213,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.7501666,...,-16.7501666+-69.7209405,2023 19:30 HORAS+01+29+19:30,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZO CRUZ - DESA...,SI,interrumpido
